In [ ]:
import os, glob, cv2, math
import numpy as np
import matplotlib.pyplot as plt

def imshow_plt(img, show_type='color'):
    plt.figure(figsize=(10,10))
    if show_type=='gray':
        plt.imshow(img, 'gray')
    else:
        plt.imshow(img[:,:,::-1])
    plt.show()
    
def distance(x1, y1, x2, y2):
    result = math.sqrt( math.pow(x1 - x2, 2) + math.pow(y1 - y2, 2))
    return result

In [ ]:
def img_crop(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(img_gray,180,255, cv2.THRESH_BINARY_INV)
#     imshow_plt(thresh, 'gray')
    
    k=np.ones((5,5), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, k)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, np.ones((15,15), np.uint8))
#     imshow_plt(thresh, 'gray')
    
    contours, hierachy = cv2.findContours(thresh, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
#     image = cv2.drawContours(img, contours, -1, (255,0,0), 5)
#     imshow_plt(image)
#     
#     print('center : ', int(img.shape[1]/2), int(img.shape[0]/2))
    for ii, cnt in enumerate(contours):
        M = cv2.moments(cnt)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
#         print(cx, cy)
#         print(int(img.shape[1]/2), int(img.shape[0]/2), cx, cy)
        d=distance(int(img.shape[1]/2), int(img.shape[0]/2), cx, cy)
        if ii==0:
            min_dist = (ii, d, cnt, (cx, cy))
        else:
#             print(cv2.contourArea(cnt))
            if d<min_dist[1] and abs(int(img.shape[1]/2)-cx)<100:# and 3000<cv2.contourArea(cnt)<1000000:
                min_dist=(ii, d, cnt, (cx, cy))
#                 image = cv2.drawContours(img, min_dist[2], -1, (255,0,0), 5)
#                 imshow_plt(image)
#     print(min_dist)
    crop_sizex = 400
    crop_sizey = 700
    cx, cy = min_dist[3][0], min_dist[3][1]
#     print(cy-crop_sizey,cy+crop_sizey,cx-crop_sizex,cx+crop_sizex)
    img_crop = img[cy-crop_sizey if cy-crop_sizey>0 else 0:cy+crop_sizey ,cx-crop_sizex if cx-crop_sizex>0 else 0:cx+crop_sizex]
#     imshow_plt(img_crop)
    
    return img_crop

In [ ]:
PATH = './03_preprocess/crop/'
label_folder = os.listdir(PATH)
label_li = [float(l.split('_')[-1]) for l in label_folder if not '.' in l]
label_folder.sort()
print('*'*30)
print('Load data')
print('*'*30)

imgs=list()
imgs_gray=list()
for label_path in label_folder:
    img_path_tmp = glob.glob(PATH+label_path+'/*')
    for path in img_path_tmp:
        print(path)
#         imgs.append(cv2.imread(path)) 
        label = label_path.split('_')[-1]
        name = path.split('\\')[-1]
        print(name+'_'+label)
        img = cv2.imread(path)
        crop_img=img_crop(img)
        imshow_plt(crop_img)
        if not os.path.isdir('./03_preprocess/crop_re/'+label_path):
            os.makedirs('./03_preprocess/crop_re/'+label_path)
        cv2.imwrite('./03_preprocess/crop_re/'+label_path+'/'+name, crop_img)


In [ ]:
def img_crop_manual(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(img_gray,180,255, cv2.THRESH_BINARY_INV)
#     imshow_plt(thresh, 'gray')
    
    k=np.ones((5,5), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, k)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, np.ones((15,15), np.uint8))
    imshow_plt(thresh, 'gray')
    
    contours, hierachy = cv2.findContours(thresh, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
#     image = cv2.drawContours(img, contours, -1, (255,0,0), 5)
#     imshow_plt(image)
#     
#     print('center : ', int(img.shape[1]/2), int(img.shape[0]/2))
    for ii, cnt in enumerate(contours):
        M = cv2.moments(cnt)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
#         print(cx, cy)
#         print(int(img.shape[1]/2), int(img.shape[0]/2), cx, cy)
        d=distance(int(img.shape[1]/2), int(img.shape[0]/2), cx, cy)
        if ii==0:
            min_dist = (ii, d, cnt, (cx, cy))
        else:
#             print(cv2.contourArea(cnt))
            if d<min_dist[1] and abs(int(img.shape[1]/2)-cx)<500:# and 3000<cv2.contourArea(cnt)<1000000:
                min_dist=(ii, d, cnt, (cx, cy))
#                 image = cv2.drawContours(img, min_dist[2], -1, (255,0,0), 5)
#                 imshow_plt(image)
#     print(min_dist)
    crop_sizex = 400
    crop_izey = 700
    cx, cy = min_dist[3][0], min_dist[3][1]
#     print(cy-crop_sizey,cy+crop_sizey,cx-crop_sizex,cx+crop_sizex)
    img_crop = img[cy-crop_sizey if cy-crop_sizey>0 else 0:cy+crop_sizey if cy+crop_sizey>0 else 0,cx-crop_sizex if cx-crop_sizex>0 else 0:cx+crop_sizex if cx+crop_sizex>0 else 0]
#     imshow_plt(img_crop)
    
    return img_crop

In [ ]:
n = '20210723_132318'

img = cv2.imread('./03_preprocess/crop/Date_pH_11.43/{}.jpg'.format(n))
imshow_plt(img)
# crop_img=img_crop_manual(img)
# imshow_plt(crop_img)
# cv2.imwrite('./03_preprocess/crop_re/Date_pH_11.43/{}.jpg'.format(n), crop_img)


crop_sizex = 400
crop_sizey = 700

cx, cy = 700,700


img_crop = img[cy-crop_sizey if cy-crop_sizey>0 else 0:cy+crop_sizey,cx-crop_sizex if cx-crop_sizex>0 else 0:cx+crop_sizex]
# print(img_crop.shape)
imshow_plt(img_crop)
cv2.imwrite('./03_preprocess/crop_re/Date_pH_11.43/{}.jpg'.format(n), img_crop)

In [ ]:
PATH = './03_preprocess/crop_re/'
label_folder = os.listdir(PATH)
label_li = [float(l.split('_')[-1]) for l in label_folder if not '.' in l]
label_folder.sort()
print('*'*30)
print('Load data')
print('*'*30)

c = 0
for label_path in label_folder:
    print(label_path)
    print(len(os.listdir(PATH+label_path)))
    c+=len(os.listdir(PATH+label_path))
    
print(c)

In [ ]:
PATH = './03_preprocess/crop_re/'
label_folder = os.listdir(PATH)
label_li = [float(l.split('_')[-1]) for l in label_folder if not '.' in l]
label_folder.sort()
print('*'*30)
print('Load data')
print('*'*30)

imgs=list()
imgs_gray=list()
for label_path in label_folder:
    print(label_path)
    if '.' in label_path:
        img_path_tmp = glob.glob(PATH+label_path+'/*')
    #     images=list()
        images=[cv2.imread(path) for path in img_path_tmp]

        plt.figure(figsize=(10,10))
        for i in range(150):
            img = images[i].astype('uint8')
#             img_mean = (img*1.0 / img.mean(axis=(0,1)))
            plt.subplot(15,10,i+1),plt.imshow(img)
    #         plt.title(titles[i])
            plt.xticks([]),plt.yticks([])

        plt.show()